
# Targeted N-Pixel Attack on Vision Transformer (ViT)

This notebook implements a **black-box adversarial attack** on a **pretrained Vision Transformer (ViT)**.
The attack generalizes the *One Pixel Attack* to an **N-pixel setting** and supports **targeted attacks**.

Key properties:
- Model: Pretrained ViT (ImageNet)
- Attack type: Black-box (no gradients)
- Optimization: Differential Evolution (DE)
- Perturbation budget: N pixels (user-defined)
- Objective: Targeted misclassification

This notebook is designed for **research, robustness evaluation, and educational purposes**.


## 1. Environment Setup

In [ ]:

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

from torchvision import models
from scipy.optimize import differential_evolution

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


## 2. Load Pretrained Vision Transformer

In [ ]:

# Load a pretrained ViT-B/16 model
model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
model.eval()
model.to(device)

# ImageNet normalization
weights = models.ViT_B_16_Weights.IMAGENET1K_V1
preprocess = weights.transforms()


## 3. Load and Preprocess an Image

In [ ]:

from PIL import Image

# Load a sample image (replace with your own image if desired)
img_path = "sample.jpg"  # user-provided image
img_pil = Image.open(img_path).convert("RGB")

img = preprocess(img_pil).unsqueeze(0).to(device)


## 4. Prediction Utility (Black-Box Access)

In [ ]:

def predict(image_tensor):
    with torch.no_grad():
        logits = model(image_tensor)
        probs = F.softmax(logits, dim=1)
    return probs.cpu().numpy()[0]


## 5. N-Pixel Perturbation Function

In [ ]:

def apply_n_pixel_attack(image, params, n_pixels):
    perturbed = image.clone()

    for i in range(n_pixels):
        base = i * 5
        x = int(params[base + 0])
        y = int(params[base + 1])
        r = params[base + 2]
        g = params[base + 3]
        b = params[base + 4]

        perturbed[0, 0, y, x] = r
        perturbed[0, 1, y, x] = g
        perturbed[0, 2, y, x] = b

    return torch.clamp(perturbed, 0, 1)


## 6. Targeted Attack Objective Function

In [ ]:

def targeted_objective(params, image, target_class, n_pixels):
    perturbed = apply_n_pixel_attack(image, params, n_pixels)
    probs = predict(perturbed)
    # Minimize negative confidence of the target class (maximize target probability)
    return -probs[target_class]


## 7. Run Targeted N-Pixel Attack

In [ ]:

# User-defined parameters
N_PIXELS = 3           # Number of pixels to modify
TARGET_CLASS = 859     # Example target class (e.g., toaster in ImageNet)

orig_probs = predict(img)
orig_pred = np.argmax(orig_probs)

print("Original prediction:", orig_pred)
print("Target class:", TARGET_CLASS)

# Image size after preprocessing (ViT default: 224x224)
H, W = img.shape[-2], img.shape[-1]

bounds = []
for _ in range(N_PIXELS):
    bounds.extend([
        (0, W - 1),  # x
        (0, H - 1),  # y
        (0, 1),      # R
        (0, 1),      # G
        (0, 1)       # B
    ])

result = differential_evolution(
    targeted_objective,
    bounds,
    args=(img, TARGET_CLASS, N_PIXELS),
    maxiter=100,
    popsize=15,
    disp=True
)

best_params = result.x
adv_img = apply_n_pixel_attack(img, best_params, N_PIXELS)

adv_probs = predict(adv_img)
adv_pred = np.argmax(adv_probs)

print("Adversarial prediction:", adv_pred)
print("Target confidence:", adv_probs[TARGET_CLASS])


## 8. Visualization

In [ ]:

def show_image(tensor, title):
    img = tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
show_image(img, f"Original (class {orig_pred})")

plt.subplot(1, 2, 2)
show_image(adv_img, f"Adversarial (class {adv_pred})")

plt.show()



## 9. Summary and Notes

- This notebook demonstrates a **targeted N-pixel black-box attack** on a pretrained ViT.
- The attack modifies a very small number of pixels while achieving targeted misclassification.
- Increasing N improves attack success but increases perceptibility and search complexity.

Possible extensions:
- Compare ViT vs CNN robustness
- Add query budget constraints
- Replace Differential Evolution with CMA-ES or NES
- Perform batch-level success rate evaluation
